# The zoo of ion channels

In [1]:
#@title NEURON install (launch only once)
# Note: Google Colab gives you access to an "instance".
# This is a virtual machine ("container" would be more correct) that runs
# Linux with already a Python installation.
!pip install neuron

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 11.5 MB/s eta 0:00:00


In [4]:
#@title Download .mod files of (some) active conductances (launch only once)
!curl -O "https://raw.githubusercontent.com/mgiugliano/ModellingNeuralSystems/refs/heads/main/notebooks/bbp_cortex_mod_files/Ca.mod"
!curl -O "https://raw.githubusercontent.com/mgiugliano/ModellingNeuralSystems/refs/heads/main/notebooks/bbp_cortex_mod_files/Ca_HVA.mod"
!curl -O "https://raw.githubusercontent.com/mgiugliano/ModellingNeuralSystems/refs/heads/main/notebooks/bbp_cortex_mod_files/Ca_LVAst.mod"
!curl -O "https://raw.githubusercontent.com/mgiugliano/ModellingNeuralSystems/refs/heads/main/notebooks/bbp_cortex_mod_files/CaDynamics_E2.mod"
!curl -O "https://raw.githubusercontent.com/mgiugliano/ModellingNeuralSystems/refs/heads/main/notebooks/bbp_cortex_mod_files/Ih.mod"
!curl -O "https://raw.githubusercontent.com/mgiugliano/ModellingNeuralSystems/refs/heads/main/notebooks/bbp_cortex_mod_files/Im.mod"
!curl -O "https://raw.githubusercontent.com/mgiugliano/ModellingNeuralSystems/refs/heads/main/notebooks/bbp_cortex_mod_files/Isinunoisy.mod"
!curl -O "https://raw.githubusercontent.com/mgiugliano/ModellingNeuralSystems/refs/heads/main/notebooks/bbp_cortex_mod_files/K_Pst.mod"
!curl -O "https://raw.githubusercontent.com/mgiugliano/ModellingNeuralSystems/refs/heads/main/notebooks/bbp_cortex_mod_files/K_Tst.mod"
!curl -O "https://raw.githubusercontent.com/mgiugliano/ModellingNeuralSystems/refs/heads/main/notebooks/bbp_cortex_mod_files/KdShu2007.mod"
!curl -O "https://raw.githubusercontent.com/mgiugliano/ModellingNeuralSystems/refs/heads/main/notebooks/bbp_cortex_mod_files/Nap_Et2.mod"
!curl -O "https://raw.githubusercontent.com/mgiugliano/ModellingNeuralSystems/refs/heads/main/notebooks/bbp_cortex_mod_files/NaTa_t.mod"
!curl -O "https://raw.githubusercontent.com/mgiugliano/ModellingNeuralSystems/refs/heads/main/notebooks/bbp_cortex_mod_files/NaTs2_t.mod"
#!curl -O "https://raw.githubusercontent.com/mgiugliano/ModellingNeuralSystems/refs/heads/main/notebooks/bbp_cortex_mod_files/ProbAMPANMDA_EMS.mod"
#!curl -O "https://raw.githubusercontent.com/mgiugliano/ModellingNeuralSystems/refs/heads/main/notebooks/bbp_cortex_mod_files/ProbGABAAB_EMS.mod"
!curl -O "https://raw.githubusercontent.com/mgiugliano/ModellingNeuralSystems/refs/heads/main/notebooks/bbp_cortex_mod_files/SK_E2.mod"
!curl -O "https://raw.githubusercontent.com/mgiugliano/ModellingNeuralSystems/refs/heads/main/notebooks/bbp_cortex_mod_files/SKv3_1.mod"
#!curl -O "https://raw.githubusercontent.com/mgiugliano/ModellingNeuralSystems/refs/heads/main/notebooks/bbp_cortex_mod_files/StochKv.mod"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1002  100  1002    0     0   4765      0 --:--:-- --:--:-- --:--:--  4748
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1022  100  1022    0     0   4908      0 --:--:-- --:--:-- --:--:--  4913
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1106  100  1106    0     0   5430      0 --:--:-- --:--:-- --:--:--  5448
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   681  100   681    0     0   3151      0 --:--:-- --:--:-- --:--:--  3138
  % Total    % Received % Xferd  Average Speed   Tim

In [6]:
#@title Compile the .mod file - launch only once and then restart the restart the Colab runtime. Go to Runtime -> Restart runtime
# Compile the .mod file. This creates a directory named x86_64.
!nrnivmodl

/content
cfiles =
Mod files: "CaDynamics_E2.mod" "Ca_HVA.mod" "Ca_LVAst.mod" "Ca.mod" "Ih.mod" "Im.mod" "Isinunoisy.mod" "KdShu2007.mod" "K_Pst.mod" "K_Tst.mod" "Nap_Et2.mod" "NaTa_t.mod" "NaTs2_t.mod" "SK_E2.mod" "SKv3_1.mod"

MODOBJS= ./CaDynamics_E2.o ./Ca_HVA.o ./Ca_LVAst.o ./Ca.o ./Ih.o ./Im.o ./Isinunoisy.o ./KdShu2007.o ./K_Pst.o ./K_Tst.o ./Nap_Et2.o ./NaTa_t.o ./NaTs2_t.o ./SK_E2.o ./SKv3_1.o
 -> Compiling mod_func.cpp
 => LINKING shared library "/content/x86_64/./libnrnmech.so"
Successfully created x86_64/special


In [7]:
#@title NEURON and other Python Libraries import
from neuron import n
import matplotlib.pyplot as plt

# run standard library for high-level simulation control functions
n.load_file("stdrun.hoc")

# Only in a jupyter notebook / jupyter lab environment
%matplotlib inline

In [8]:
#@title Define and create the model
n.celsius = 30

# create model
soma  = n.Section(name="soma")
soma.cm = 1      # capacitance (uF/cm^2)
soma.L  = 20     # length µm
soma.diam = 20   # diameter µm

apc = n.APCount(0.5, sec=soma)  #

soma.insert('pas')      # add passive properties
soma.g_pas = 1/10000    # set the specific membrane

# Insert ALL the mechanisms provided
soma.insert('pas')         # Passive leak (built-in)
soma.insert('NaTa_t')      # Transient Na+
soma.insert('NaTs2_t')     # Shifted transient Na+
soma.insert('Nap_Et2')     # Persistent Na+
soma.insert('K_Tst')       # Transient K+
soma.insert('K_Pst')       # Persistent K+
soma.insert('Im')          # M-type K+
soma.insert('Ca')          # HVA Ca2+
soma.insert('Ca_LVAst')    # LVA Ca2+
soma.insert('Ih')          # HCN current
soma.insert('CaDynamics_E2') # Calcium dynamics (vital for Ca2+ channels)

# Set reversal potentials (good practice)
soma.ena = 50
soma.ek = -85
soma.eca = 120
soma.e_pas = -70
soma.ehcn = -45 # Reversal for Ih is set in its own .mod file

# current clamp
stim = n.IClamp(soma(0.5))
stim.delay = 10         # start of the current injection (ms)
stim.dur  = 2           # duration (ms)
stim.amp  = 0.035        # amplitude (nA)

stim2 = n.IClamp(soma(0.5))
stim2.delay = 100         # start of the current injection (ms)
stim2.dur  = 175           # duration (ms)
stim2.amp  = 0.01        # amplitude (nA)


# record voltage at the soma, the injected current, and the time
soma_v = n.Vector()
soma_v.record(soma(0.5)._ref_v)

stim_current = n.Vector()
stim_current.record(stim._ref_i)
stim_current1 = n.Vector()
stim_current1.record(stim2._ref_i)

time = n.Vector()
time.record(n._ref_t)

Nspikes = n.h.Vector()
Nspikes.record(apc._ref_n)

n.t = 0
n.tstop = 300 # set the simulation time
n.dt = 0.025
n.v_init = -70

n.run()


# plotting
f, (ax0, ax1) = plt.subplots(2,1, gridspec_kw = {'height_ratios':[4, 1]})
ax0.plot(time,soma_v, 'k')
ax1.plot(time,stim_current+stim_current1, 'gray', label='I (nA)')
ax0.set_ylabel('Memrabe Potential (mV)')
ax1.set_xlabel('Time (ms)')
ax1.set_ylabel('Current Injected (nA)')
ax0.set_ylim([-90, 50])

ValueError: argument not a density mechanism name.

In [3]:
!ls -l x86_64/

total 1696
-rw-r--r-- 1 root root  21863 Oct 23 15:20 Ca.cpp
-rw-r--r-- 1 root root  18953 Oct 23 15:20 CaDynamics_E2.cpp
-rw-r--r-- 1 root root  62400 Oct 23 15:23 CaDynamics_E2.o
-rw-r--r-- 1 root root  21970 Oct 23 15:20 Ca_HVA.cpp
-rw-r--r-- 1 root root  67696 Oct 23 15:23 Ca_HVA.o
-rw-r--r-- 1 root root  21446 Oct 23 15:20 Ca_LVAst.cpp
-rw-r--r-- 1 root root  64568 Oct 23 15:23 Ca_LVAst.o
-rw-r--r-- 1 root root  67664 Oct 23 15:23 Ca.o
-rw-r--r-- 1 root root  18703 Oct 23 15:20 Ih.cpp
-rw-r--r-- 1 root root  49728 Oct 23 15:23 Ih.o
-rw-r--r-- 1 root root  20115 Oct 23 15:20 Im.cpp
-rw-r--r-- 1 root root  61296 Oct 23 15:23 Im.o
-rw-r--r-- 1 root root      0 Oct 23 15:20 Isinunoisy.cpp
-rw-r--r-- 1 root root    936 Oct 23 15:23 Isinunoisy.o
-rw-r--r-- 1 root root  21666 Oct 23 15:20 KdShu2007.cpp
-rw-r--r-- 1 root root  63528 Oct 23 15:23 KdShu2007.o
-rw-r--r-- 1 root root  21531 Oct 23 15:20 K_Pst.cpp
-rw-r--r-- 1 root root  65192 Oct 23 15:23 K_Pst.o
-rw-r--r-- 1 root root  21285